In [ ]:
base_dir = '/home/labs/hornsteinlab/Collaboration/FUNOVA/input/images/raw'
NOVA_HOME = '/home/labs/hornsteinlab/Collaboration/NOVA_GAL/NOVA'
FILE_EXTENSION = '.tiff'
KEY_BATCH = 'Batch'

In [ ]:
import os
import sys
os.environ['NOVA_HOME'] = NOVA_HOME
sys.path.insert(1, os.getenv('NOVA_HOME'))
from tools.show_images_utils import *
%load_ext autoreload
%autoreload 2

In [ ]:
df = extract_image_metadata(base_dir, FILE_EXTENSION = FILE_EXTENSION, KEY_BATCH = KEY_BATCH)
df

In [ ]:
for col in ['Marker', 'Condition', 'CellLine', 'Rep', 'Batch', 'Panel']:
    print(col, 'values are:', np.unique(df[col]))

In [ ]:
show_images(df, marker = 'Senescence_signaling', show_DAPI = True, 
            batch = 1, cell_line ='Control_1048087', 
            condition = 'stress', rep=2, funova = True, samples = 10)

In [ ]:
show_images(df, marker = 'Senescence_signaling', show_DAPI = True, 
            batch = 1, cell_line ='Control_1048087', 
            condition = 'stress', rep=2, funova = True, samples = 10)

In [ ]:
# Example Usage:

# Show images for the marker 'P54'
# Filters: default parameters for all other options
# show_images(df, 'P54')

# Show images for the marker 'G3BP1'
# Exclude DAPI, limit to 3 samples, and filter by Batch 1, Rep 1, WT cell line, and Untreated condition
# show_images(df, marker='G3BP1', show_DAPI=False, samples=3, batch=1, rep=1, cell_line='WT', condition='Untreated')

# get df of specific images
# get_specific_imgs(df, marker='G3BP1', show_DAPI=False, samples=3, batch=1, rep=1, cell_line='WT', condition='Untreated')

# Create a PDF report for the marker 'TIA1' under the 'Untreated' condition
# Output the report as 'marker_name_Untreated/stress_Report.pdf'
# create_img_pdf_report(df, marker='TIA1', condition='Untreated', output_file='Untreated_Report.pdf')

In [ ]:
# show_images(df, marker = 'DAPI', show_DAPI = False, batch = 2, panel='L', rep=1, funova = True, samples = 10)

In [ ]:
show_images(df, marker = 'DAPI', show_DAPI = False, batch = 2, cell_line ='C9orf72-HRE' , funova = True, samples = 10)

In [ ]:
show_images(df, marker='DAPI', show_DAPI=False, batch=2, panel='L', cell_line='C9orf72-HRE', rep=1,
            condition='stress', funova=True, samples=30)

In [ ]:
show_images(df, marker='DAPI', show_DAPI=False, batch=2, panel='L', cell_line='TDP--43-G348V', rep=1,
            condition='stress', funova=True, samples=30)

In [ ]:
show_images(df, marker='Protein_degradation', show_DAPI=False, batch=2, panel='L', cell_line='C9orf72-HRE', rep=1,
            condition='stress', funova=True, samples=30)

In [ ]:
show_images(df, marker='Senescence_signaling', show_DAPI=False, batch=2, panel='L', cell_line='C9orf72-HRE', rep=1,
            condition='stress', funova=True, samples=10)

In [ ]:
# for marker in ['Nuclear_speckles_SON', 'Necroptosis_HMGB1', 'Splicing_factories', 'Ubiquitin_levels', 'UPR_ATF6',
#         'Parthanatos_late', 'Parthanatos_early', 'Cytoskeleton'%]:
#     for condition in ['Untreated', 'stress']:
#         create_img_pdf_report(df, marker=marker, condition=condition, output_file=f'{condition}_Report.pdf',
#                      reps=2, batches=4)